**Q1**

In [ ]:
import math
from collections import Counter, defaultdict

# Data Loading and Preprocessing
def read_file(path):
    with open(path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f if line.strip()]

def tokenize_sentences(sentences):
    return [['<s>'] + s.split() + ['</s>'] for s in sentences]

def load_and_tokenize(path):
    return tokenize_sentences(read_file(path))

# Unigram Model with Dirichlet Smoothing 
class UnigramModel:
    def __init__(self, alpha=0.1):
        self.alpha = alpha
        self.counts = Counter()
        self.total = 0
        self.vocab = set()

    def train(self, corpus):
        for sentence in corpus:
            self.counts.update(sentence)
        self.total = sum(self.counts.values())
        self.vocab = set(self.counts.keys())

    def prob(self, word):
        return (self.counts[word] + self.alpha) / (self.total + self.alpha * len(self.vocab))

    def perplexity(self, corpus):
        log_prob = 0
        word_count = 0
        for sentence in corpus:
            for word in sentence:
                log_prob += math.log(self.prob(word))
                word_count += 1
        return math.exp(-log_prob / word_count)

# N-Gram Model with Kneser-Ney Smoothing 
class NGramModelKN:
    def __init__(self, n, discount=0.75):
        self.n = n
        self.discount = discount
        self.ngram_counts = Counter()
        self.prefix_counts = Counter()
        self.continuation_counts = defaultdict(set)
        self.unique_continuations = defaultdict(int)
        self.lower_order_model = None
        self.total_unique_ngrams = 0
        self.total_vocab = set()

    def train(self, corpus):
        for sentence in corpus:
            tokens = ['<s>'] * (self.n - 1) + sentence + ['</s>']
            for i in range(len(tokens) - self.n + 1):
                ngram = tuple(tokens[i:i + self.n])
                prefix = ngram[:-1]
                word = ngram[-1]
                self.ngram_counts[ngram] += 1
                self.prefix_counts[prefix] += 1
                self.continuation_counts[word].add(prefix)
                self.total_vocab.add(word)
        self.total_unique_ngrams = len(self.ngram_counts)

        #  Precompute how many unique words follow each prefix
        for ngram in self.ngram_counts:
            prefix = ngram[:-1]
            self.unique_continuations[prefix] += 1

        if self.n > 1:
            self.lower_order_model = NGramModelKN(self.n - 1, self.discount)
            self.lower_order_model.train(corpus)

    def prob(self, ngram):
        if self.n == 1:
            word = ngram[0]
            return len(self.continuation_counts[word]) / self.total_unique_ngrams if self.total_unique_ngrams > 0 else 1e-10

        prefix, word = tuple(ngram[:-1]), ngram[-1]
        count_ngram = self.ngram_counts[tuple(ngram)]
        count_prefix = self.prefix_counts[prefix]

        if count_prefix > 0:
            lambda_factor = self.discount * self.unique_continuations[prefix] / count_prefix
            lower_order_prob = self.lower_order_model.prob(ngram[1:])
            return max(count_ngram - self.discount, 0) / count_prefix + lambda_factor * lower_order_prob
        else:
            return self.lower_order_model.prob(ngram[1:])

    def perplexity(self, corpus):
        log_prob = 0
        word_count = 0
        for sentence in corpus:
            tokens = ['<s>'] * (self.n - 1) + sentence + ['</s>']
            for i in range(self.n - 1, len(tokens)):
                ngram = tokens[i - self.n + 1:i + 1]
                p = self.prob(ngram)
                log_prob += math.log(p if p > 0 else 1e-10)
                word_count += 1
        return math.exp(-log_prob / word_count)


# Main Execution 
if __name__ == "__main__":
    train = load_and_tokenize('/content/drive/MyDrive/Colab Notebooks/train.csv')
    val = load_and_tokenize('/content/drive/MyDrive/Colab Notebooks/val.csv')
    test = load_and_tokenize('/content/drive/MyDrive/Colab Notebooks/test.csv')

    # Unigram model
    unigram = UnigramModel(alpha=0.1)
    unigram.train(train)
    print("Unigram Perplexity:", unigram.perplexity(test))

    # Bigram model with Kneser-Ney
    bigram = NGramModelKN(n=2)
    bigram.train(train)
    print("Bigram (KN) Perplexity:", bigram.perplexity(test))

    # Trigram model with Kneser-Ney
    trigram = NGramModelKN(n=3)
    trigram.train(train)
    print("Trigram (KN) Perplexity:", trigram.perplexity(test))

Unigram Perplexity: 1065.285048562787
Bigram (KN) Perplexity: 254.23441560040953
Trigram (KN) Perplexity: 210.85815121363115


In [ ]:
# Hyperparameter Tuning 

def tune_unigram_alpha(train, val, alphas):
    best_alpha = None
    best_perplexity = float('inf')
    for alpha in alphas:
        model = UnigramModel(alpha=alpha)
        model.train(train)
        ppl = model.perplexity(val)
        print(f"Alpha={alpha:.4f} -> Val Perplexity={ppl:.2f}")
        if ppl < best_perplexity:
            best_perplexity = ppl
            best_alpha = alpha
    return best_alpha, best_perplexity

def tune_discount_ngram(train, val, n, discounts):
    best_discount = None
    best_perplexity = float('inf')
    for d in discounts:
        model = NGramModelKN(n=n, discount=d)
        model.train(train)
        ppl = model.perplexity(val)
        print(f"Discount={d:.2f} (n={n}) -> Val Perplexity={ppl:.2f}")
        if ppl < best_perplexity:
            best_perplexity = ppl
            best_discount = d
    return best_discount, best_perplexity

# Run Tuning 
print("\nTuning Unigram Alpha:")
alpha_candidates = [0.01, 0.05, 0.1, 0.2, 0.5, 1.0]
best_alpha, best_unigram_ppl = tune_unigram_alpha(train, val, alpha_candidates)
print(f"\nBest Alpha: {best_alpha:.4f} with Val Perplexity: {best_unigram_ppl:.2f}")

print("\nTuning Bigram Discount:")
discount_candidates = [0.1,0.2,0.35,0.45, 0.5,0.6, 0.75,0.9, 1.0]
best_bigram_discount, best_bigram_ppl = tune_discount_ngram(train, val, 2, discount_candidates)
print(f"\nBest Bigram Discount: {best_bigram_discount:.2f} with Val Perplexity: {best_bigram_ppl:.2f}")

print("\nTuning Trigram Discount:")
best_trigram_discount, best_trigram_ppl = tune_discount_ngram(train, val, 3, discount_candidates)
print(f"\nBest Trigram Discount: {best_trigram_discount:.2f} with Val Perplexity: {best_trigram_ppl:.2f}")



Tuning Unigram Alpha:
Alpha=0.0100 -> Val Perplexity=1150.65
Alpha=0.0500 -> Val Perplexity=1089.19
Alpha=0.1000 -> Val Perplexity=1064.47
Alpha=0.2000 -> Val Perplexity=1041.47
Alpha=0.5000 -> Val Perplexity=1016.14
Alpha=1.0000 -> Val Perplexity=1005.28

Best Alpha: 1.0000 with Val Perplexity: 1005.28

Tuning Bigram Discount:
Discount=0.10 (n=2) -> Val Perplexity=304.93
Discount=0.20 (n=2) -> Val Perplexity=279.15
Discount=0.35 (n=2) -> Val Perplexity=262.91
Discount=0.45 (n=2) -> Val Perplexity=257.69
Discount=0.50 (n=2) -> Val Perplexity=256.09
Discount=0.60 (n=2) -> Val Perplexity=254.54
Discount=0.75 (n=2) -> Val Perplexity=256.38
Discount=0.90 (n=2) -> Val Perplexity=267.03
Discount=1.00 (n=2) -> Val Perplexity=364.36

Best Bigram Discount: 0.60 with Val Perplexity: 254.54

Tuning Trigram Discount:
Discount=0.10 (n=3) -> Val Perplexity=378.53
Discount=0.20 (n=3) -> Val Perplexity=300.48
Discount=0.35 (n=3) -> Val Perplexity=252.59
Discount=0.45 (n=3) -> Val Perplexity=235.63
Di